# Generating images with MX-Font model from a reference style
In this example we'll generate images with trained MX-Font model from a reference style.
If you want to generate multiple styles, please check using `eval.py` instead of using this example file (because it is much simpler to load the referece styles).

### 1. Loading packages
* First, load the packages used in this code.
* All of the packages are avilable in `pip`.

In [1]:
import json
from pathlib import Path
from PIL import Image

import torch
from sconf import Config
from torchvision import transforms

* These modules are defined in this repository.

In [2]:
import models
from datasets import read_font, render
from utils import save_tensor_to_image

### 2. Build model
* Build and load the trained model.
* `weight_path` : 
    * The location of the trained model weight.

In [7]:
########################################################
weight_path = "last.pth"  # path to weight to infer
########################################################

cfg = Config("cfgs/eval.yaml", default="cfgs/defaults.yaml")
transform = transforms.Compose(
    [transforms.Resize((128, 128)), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
)
decomposition = json.load(open("data/kz_decomposition.json", encoding='utf-8'))

g_kwargs = cfg.get('g_args', {})
gen = models.Generator(1, cfg.C, 1, **g_kwargs).cuda().eval()
weight = torch.load(weight_path)
if "generator_ema" in weight:
    weight = weight["generator_ema"]
gen.load_state_dict(weight)

<All keys matched successfully>

### 3. Load reference images.
* `ref_path`: 
    * The path of reference font or images.
    * If you are using a ttf file, set this to the location of the ttf file.
    * If you want to use rendered images, set this to the path to the directory which contains the reference images.
* `extension`:
    * If you are using image files, set this to their extension(png, jpg, etc..). 
    * This will be ignored if `use_ttf` is True.
* `batch_size`:
    * The number of images inferred at once.

In [11]:
########################################################
ref_path = "data/images/test/arial"  # Path to the reference images
extension = "png"  # Extension of the reference images
batch_size = 1  # The batch size
########################################################

ref_paths = Path(ref_path).glob(f"*.{extension}")
ref_imgs = torch.stack([transform(Image.open(str(p))) for p in ref_paths]).cuda()
ref_batches = torch.split(ref_imgs, batch_size)

### 4. Extract style factors from reference images.

In [12]:
style_facts = {}

for batch in ref_batches:
    style_fact = gen.factorize(gen.encode(batch), 0)
    for k in style_fact:
        style_facts.setdefault(k, []).append(style_fact[k])
        
style_facts = {k: torch.cat(v).mean(0, keepdim=True) for k, v in style_facts.items()}

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 3.28 GiB is allocated by PyTorch, and 182.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### 5. Generate the images.
* `gen_chars`: The characters to generate.
* `save_dir`: Path to save the generated images.
* `source_path`: Path to the font file used as the source font.

In [10]:
english_chars_digit = list(range(65, 91)) + list(range(97, 123)) + list(range(48, 58))  # A-Z, a-z, 0-9
russian_chars = list(range(0x0410, 0x0450))+[0x0401, 0x0451]  # А-я
kazakh_chars = [0x04D8, 0x04D9, 0x0492, 0x0493, 0x049A, 0x049B, 0x04A2, 0x04A3, 0x04E8, 0x04E9, 0x04B0, 0x04B1, 0x04AE, 0x04AF, 0x04BA, 0x04BB, 0x0406, 0x0456]
########################################################
gen_chars =  [chr(i) for i in kazakh_chars]  # Characters to generate
save_dir = Path("./results4")  # Directory where you want to save generated images
source_path = r"data\ttfs\val\EF Circular Bold.ttf"  # Path to the font file to render the source images
########################################################

save_dir.mkdir(parents=True, exist_ok=True)

source_font = read_font(source_path)
for char in gen_chars:
    source_img = transform(render(source_font, char)).unsqueeze(0).cuda()
    char_facts = gen.factorize(gen.encode(source_img), 1)
    
    gen_feats = gen.defactorize([style_facts, char_facts])
    out = gen.decode(gen_feats).detach().cpu()[0]

    prefix = 'upper_' if char.isupper() else 'lower_'
    path = save_dir / f"{prefix}{char}.png"
    save_tensor_to_image(out, path)

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 3.30 GiB is allocated by PyTorch, and 159.92 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)